### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.

In [8]:
import os
import json
import datetime

lab_prefix="ais-pevo3-"
tool_prefix="st-pevo1-"
deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"{lab_prefix}{deployment_name}" # change the name to match your naming style

def log(stdout, name, action):
    if stdout.n.startswith("ERROR"):
        print("👎🏻 ", name, " was NOT ", action, ": ", stdout)
    else:
        print("👍🏻 ", name, " was ", action, " ⌚ ", datetime.datetime.now().time())

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} -o json  
#print(deployment_stdout) 
if deployment_stdout.n.startswith("ERROR"):
    print("👎🏻 No active deployments: \n", deployment_stdout)
    raise KeyboardInterrupt 

deployment = json.loads(deployment_stdout.n)
if deployment.get("properties").get("outputResources") is None:
    delete_stdout = ! az group delete --name {resource_group_name} -y
    log(delete_stdout, resource_group_name, "deleted")
    print("👎🏻 Failed deployment: \n", deployment)
    print("\nexiting without doing any purge of resources")
    raise KeyboardInterrupt    

for resource in deployment.get("properties").get("outputResources"):
    resource_id = resource.get("id")
    try:
        query = "\"{type:type, name:name, location:location}\""
        resource_stdout = ! az resource show --id {resource_id} --query {query} -o json
        resource = json.loads(resource_stdout.n)
        resource_name = resource.get("name")
        resource_type = resource.get("type")  
        if resource_type == "Microsoft.CognitiveServices/accounts":
            resource_location = "\"" + resource.get("location") + "\""
            delete_stdout = ! az cognitiveservices account delete -g {resource_group_name} -n {resource_name}
            log(delete_stdout, resource_name, "deleted")
            delete_stdout = ! az cognitiveservices account purge -g {resource_group_name} -n {resource_name} -l {resource_location}
            log(delete_stdout, resource_name, "purged")
        elif resource_type == "Microsoft.ApiManagement/service":
            resource_location = "\"" + resource.get("location") + "\""
            delete_stdout = ! az apim delete -n {resource_name} -g {resource_group_name} -y
            log(delete_stdout, resource_name, "deleted")
            delete_stdout = ! az apim deletedservice purge --service-name {resource_name} --location {resource_location}
            log(delete_stdout, resource_name, "purged")
    except:
        print("✌🏻 ", resource_id, " ignored")
delete_stdout = ! az group delete --name {resource_group_name} -y
log(delete_stdout, resource_group_name, "deleted")



👍🏻  ais-pevo3-semantic-caching  was  deleted  ⌚  15:14:03.427507
👎🏻 Failed deployment: 
 {'id': '/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/ais-pevo3-semantic-caching/providers/Microsoft.Resources/deployments/semantic-caching', 'location': None, 'name': 'semantic-caching', 'properties': {'correlationId': '97a9bdea-1afc-4443-bb93-a6045152d982', 'debugSetting': None, 'dependencies': [{'dependsOn': [{'id': '/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/ais-pevo3-semantic-caching/providers/Microsoft.Cache/redisEnterprise/rediscache-dvr2i4cguab6u', 'resourceGroup': 'ais-pevo3-semantic-caching', 'resourceName': 'rediscache-dvr2i4cguab6u', 'resourceType': 'Microsoft.Cache/redisEnterprise'}], 'id': '/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/ais-pevo3-semantic-caching/providers/Microsoft.Cache/redisEnterprise/rediscache-dvr2i4cguab6u/databases/default', 'resourceGroup': 'ais-pevo3-semantic-caching', 'resourceName': 'rediscach

KeyboardInterrupt: 